In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import rc
rc('font', family='AppleGothic')
%matplotlib inline
import seaborn as sns
import pandas_profiling as pp
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
#plt.rc('font', family='NanumGothic', size= 20)

data1=pd.read_excel('master_data분석용.xlsx')
data=data1

In [12]:
data.columns

Index(['펀딩시작날짜', '펀딩마감날짜', '펀딩기간일', '펀딩시작월', '펀딩시작요일', '목표금액', '좋아요수',
       '리워드종류수', '이미지수', '비디오수', '마감배송차이', '인스타존재여부', '인스타팔로워수', '와디즈팔로워수',
       '과거프로젝트수', '과거성공프로젝트수', '앵콜펀딩여부', '음절수', '단어수', '문장수', 'Strong',
       'Under', '가독성', '가독성2', '제목단어수', '펀딩성공여부', '달성률', '카테고리', '글업데이트수',
       '일펀딩금액', '일글수', '문장당강조', '문장당밑줄'],
      dtype='object')

# PREPROCESSING

In [2]:
# 이상치 제거

# 달성률>5000% 제거
# 펀딩기간일 0 - 2개제거
# 마감배송일 -1넘어가면 로그취했을 때 inf 발생

data=data[(data.펀딩기간일!=0)&(data.펀딩기간일<75)]
data=data[data.달성률<5000]
data=data[data['마감배송차이']>-1]

In [3]:
data['펀딩기간일_T']=preprocessing.scale(np.log(data['펀딩기간일']+1))
data['Strong_T']=preprocessing.scale(np.log(data['Strong']+1))
data['Under_T']=preprocessing.scale(np.log(data['Under']+1))
data['문장수_T']=preprocessing.scale(np.log(data['문장수']+1))
data['달성률_T']=preprocessing.scale(np.log(data['달성률']+1))
data['리워드종류수_T']=preprocessing.scale(np.log(np.log(data['리워드종류수']+1))+2)
data['마감배송차이_T']=preprocessing.scale(np.log(data['마감배송차이']+1))
data['와디즈기사개수_T']=preprocessing.scale(np.log(data['와디즈기사개수']+1))
data['가독성_T']=RobustScaler().fit_transform(data.가독성.values.reshape(-1, 1))
data['가독성2_T']=preprocessing.scale(np.log(data['가독성2']+1))
data['이미지수_T']=preprocessing.scale(np.log(data['이미지수']+1))

In [33]:
#q1, q2, q3 = np.percentile(data.달성률, [25, 50, 75])
#print(q1,q2,q3)

107.0 281.0 877.0


In [36]:
#print(len(data[data.달성률<100]))
#print(len(data[(data.달성률<300)&(data.달성률>=100)]))
#print(len(data[(data.달성률<900)&(data.달성률>=300)]))
#print(len(data[(data.달성률<5000)&(data.달성률>=900)]))

1115
1594
1276
1109


# ENCODING - Categorial Variables

In [4]:
# 펀딩시작월 mapping
펀딩시작월_dict = {1:'Jan', 2:'Feb', 3:'Mar',4:'Apr',5:'May',6:'June',7:'July',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
data['펀딩시작월'] = data.펀딩시작월.map(펀딩시작월_dict)

In [5]:
categorical_feature_names= ['앵콜펀딩여부','펀딩성공여부']
for col in categorical_feature_names:
    data[col] = data[col].astype("category")

In [6]:
# Multi변수 명목변수화-원핫인코딩
펀딩시작월 = pd.get_dummies(data['펀딩시작월'])
data=pd.concat([data, 펀딩시작월], axis=1)
del data['펀딩시작월']

카테고리 = pd.get_dummies(data['카테고리'])
data=pd.concat([data, 카테고리], axis=1)
del data['카테고리']

펀딩시작요일 = pd.get_dummies(data['펀딩시작요일'])
data=pd.concat([data, 펀딩시작요일], axis=1)
del data['펀딩시작요일']

In [20]:
feature_names=['와디즈기사개수_T',
               #'좋아요수',
               '목표금액', 
               '리워드종류수_T',
               '이미지수_T',
               '비디오수',
               '마감배송차이_T',
               '와디즈팔로워수',
               '과거프로젝트수',
               '과거성공프로젝트수',
               '앵콜펀딩여부',
               #'음절수',
               #'단어수',
               '문장수_T',
               'Strong_T',
               'Under_T',
               '가독성_T',
               '가독성2_T', 
               '제목단어수', 
               '펀딩성공여부',
               '디자인소품',
               '반려동물',
               '뷰티',
               '스포츠·모빌리티',
              '여행·레저',
               '테크·가전',
               '패션·잡화',
               '푸드', 
               '홈리빙',
               'Fri',
               'Mon',
               'Sat',
               'Sun',
               'Thu',
               'Tue',
               'Wed',
               '펀딩기간일_T',
               '일펀딩금액',
               '일글수',
              '문장당강조',
               '문장당밑줄',
              'Jan',
              'Feb',
              'Mar',
              'Apr',
              'May',
              'June',
              'July',
              'Aug',
              'Sep',
              'Oct',
              'Nov',
              'Dec']
data_f = data[feature_names]

# REGRESSION

In [21]:
X = data_f
y = data.달성률_T

# 학습용/테스트용 데이터 분리

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [22]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000)

rf.fit(X_train, y_train)

print('훈련셋점수:',rf.score(X_train, y_train))
print('테스트셋점수:',rf.score(X_test, y_test))

y_pred = rf.predict(X_test)

import sklearn.metrics as mt

#평가 지표 값들을 계산한다.
mse = mt.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = mt.r2_score(y_test, y_pred)
print("RF선형MSE:{:.3f}\nRMSE: {:.3f}\nRF선형R2: {:.3f}".format(mse,rmse,r2))

#수정결정계수
n = len(X_train)
k = len(X_train.columns)
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
print('-'*10)
print('RF선형회귀Adjusted R2: {:.4f}'.format(adj_r2))
#print(mt.accuracy_score( y_pred, y_test))
#print(mt.confusion_matrix(y_pred, y_test))

훈련셋점수: 0.9604832529750871
테스트셋점수: 0.7244522987244325
RF선형MSE:0.295
RMSE: 0.543
RF선형R2: 0.724
----------
RF선형회귀Adjusted R2: 0.7205


In [23]:
from sklearn.model_selection import GridSearchCV

params_rf = {
    'n_estimators': [100, 300],
    'max_depth': [3, 6], 
    'class_weight': [None, 'balanced']}

grid_rf = GridSearchCV(rf, params_rf, 
                      cv = 5, n_jobs = -1, 
                      scoring = 'accuracy')

grid_rf.fit(X_train, y_train)
grid_rf_pred = grid_rf.predict(X_test)
print(grid_rf.best_estimator_)
print('accuracy', mt.accuracy_score(y_test,grid_rf_pred) )
print('precision', mt.precision_score(y_test,grid_rf_pred) )
print(mt.classification_report(y_test,grid_rf_pred))
print(mt.confusion_matrix(y_test,grid_rf_pred))

pd.DataFrame({'속성명':X.columns, '중요도':rf.feature_importances_}).sort_values('중요도',ascending=False)

ValueError: Invalid parameter class_weight for estimator RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False). Check the list of available parameters with `estimator.get_params().keys()`.

In [14]:
#다중선형회귀 실시
import sklearn.linear_model as lm
import sklearn.metrics as mt
import numpy as np

reg=lm.LinearRegression().fit(X_train,y_train)
y_pred = reg.predict(X_test)

#오차 및 결정계수
mse = mt.mean_squared_error(y_test, y_pred) # 테스트예측값
rmse = np.sqrt(mse)
r2 = mt.r2_score(y_test, y_pred)
print('선형회귀MSE: {:3f}\nRMSE: {:.3f}\n선형회귀R2: {:.3f}'.format(mse, rmse, r2))

#수정결정계수
n = len(X_train)
k = len(X_train.columns)
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
print('-'*10)
print('선형회귀 Adjusted R2: {:.4f}'.format(adj_r2))

#coefs = pd.Series(data=np.round(reg.coef_,3),index=features_.columns)
#coefs_sort = coefs.sort_values(ascending=False)
#print(coefs_sort)
#print('절편:', round(reg.intercept_, 3))
#np.set_printoptions(suppress=False)

선형회귀MSE: 0.325095
RMSE: 0.570
선형회귀R2: 0.696
----------
선형회귀 Adjusted R2: 0.6921


In [175]:
#릿지 회귀 실시
ridge=lm.Ridge(alpha = 0.01)
reg=ridge.fit(X_train,y_train)
y_pred = reg.predict(X_test)

r2=mt.r2_score(y_test,y_pred)
print('릿지회귀 R2: {:.4f}'.format(r2))
print('릿지회귀 Adjusted R2: {:.4f}'.format(adj_r2))

#오차 및 결정계수
mse = mt.mean_squared_error(y_test, y_pred) # 테스트예측값
rmse = np.sqrt(mse)
r2 = mt.r2_score(y_test, y_pred)
print('릿지MSE: {:3f}\nRMSE: {:.3f}\n릿지R2: {:.3f}'.format(mse, rmse, r2))

#수정결정계수
n = len(X_train)
k = len(X_train.columns)
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
print('-'*10)
print('릿지Adjusted R2: {:.4f}'.format(adj_r2))

릿지회귀 R2: 0.6964
릿지회귀 Adjusted R2: 0.6921
릿지MSE: 0.325095
RMSE: 0.570
릿지R2: 0.696
----------
릿지Adjusted R2: 0.6921


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.00427e-19): result may not be accurate.
  overwrite_a=True).T


In [176]:
#라쏘 회귀 실시
lasso=lm.Lasso(alpha = 0.1)
reg=ridge.fit(X_train,y_train)
y_pred = reg.predict(X_test)

r2=mt.r2_score(y_test,y_pred)
print('라쏘회귀 R2: {:.4f}'.format(r2))
print('라쏘회귀 Adjusted R2: {:.4f}'.format(adj_r2))

#오차 및 결정계수
mse = mt.mean_squared_error(y_test, y_pred) # 테스트예측값
rmse = np.sqrt(mse)
r2 = mt.r2_score(y_test, y_pred)
print('라쏘회귀MSE: {:3f}\nRMSE: {:.3f}\n라쏘회귀R2: {:.3f}'.format(mse, rmse, r2))
#수정결정계수
n = len(X_train)
k = len(X_train.columns)
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
print('-'*10)
print('라쏘회귀 Adjusted R2: {:.4f}'.format(adj_r2))


라쏘회귀 R2: 0.6964
라쏘회귀 Adjusted R2: 0.6921
라쏘회귀MSE: 0.325095
RMSE: 0.570
라쏘회귀R2: 0.696
----------
라쏘회귀 Adjusted R2: 0.6921


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.00427e-19): result may not be accurate.
  overwrite_a=True).T


In [178]:
#엘라스틱 회귀 실시
elastic = lm.ElasticNet(alpha = 0.01, l1_ratio=0.5)
reg=ridge.fit(X_train,y_train)
y_pred = reg.predict(X_test)

r2=mt.r2_score(y_test,y_pred)

print('엘라스틱 R2: {:.4f}'.format(r2))
print('엘라스틱 Adjusted R2: {:.4f}'.format(adj_r2))

#오차 및 결정계수
mse = mt.mean_squared_error(y_test, y_pred) # 테스트예측값
rmse = np.sqrt(mse)
r2 = mt.r2_score(y_test, y_pred)
print('엘라스틱MSE: {:3f}\nRMSE: {:.3f}\n엘라스틱R2: {:.3f}'.format(mse, rmse, r2))
#수정결정계수
n = len(X_train)
k = len(X_train.columns)
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
print('-'*10)
print('엘라스틱Adjusted R2: {:.4f}'.format(adj_r2))


엘라스틱 R2: 0.6964
엘라스틱 Adjusted R2: 0.6921
엘라스틱MSE: 0.325095
RMSE: 0.570
엘라스틱R2: 0.696
----------
엘라스틱Adjusted R2: 0.6921


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.00427e-19): result may not be accurate.
  overwrite_a=True).T


# 변수추가

In [7]:
data['시작1일차이_T']=preprocessing.scale(np.log(data['시작1일차이']+1))
data['시작5일차이_T']=preprocessing.scale(np.log(data['시작5일차이']+1))
data['시작10일차이_T']=preprocessing.scale(np.log(data['시작10일차이']+1))
data['시작15일차이_T']=preprocessing.scale(np.log(data['시작15일차이']+1))
data['시작20일차이_T']=preprocessing.scale(np.log(data['시작20일차이']+1))
data['시작25일차이_T']=preprocessing.scale(np.log(data['시작25일차이']+1))
data['마감1일차이_T']=preprocessing.scale(np.log(data['마감1일차이']+1))
data['마감5일차이_T']=preprocessing.scale(np.log(data['마감5일차이']+1))
data['마감10일차이_T']=preprocessing.scale(np.log(data['마감10일차이']+1))
data['마감15일차이_T']=preprocessing.scale(np.log(data['마감15일차이']+1))
data['마감20일차이_T']=preprocessing.scale(np.log(data['마감20일차이']+1))
data['마감25일차이_T']=preprocessing.scale(np.log(data['마감25일차이']+1))

In [8]:
feature_names=['와디즈기사개수_T',
               #'좋아요수',
               '목표금액', 
               '리워드종류수_T',
               '이미지수_T',
               '비디오수',
               '마감배송차이_T',
               '와디즈팔로워수',
               '과거프로젝트수',
               '과거성공프로젝트수',
               '앵콜펀딩여부',
               #'음절수',
               #'단어수',
               '문장수_T',
               'Strong_T',
               'Under_T',
               '가독성_T',
               '가독성2_T', 
               '제목단어수', 
               '펀딩성공여부',
               '디자인소품',
               '반려동물',
               '뷰티',
               '스포츠·모빌리티',
              '여행·레저',
               '테크·가전',
               '패션·잡화',
               '푸드', 
               '홈리빙',
               'Fri',
               'Mon',
               'Sat',
               'Sun',
               'Thu',
               'Tue',
               'Wed',
               '펀딩기간일_T',
               '일펀딩금액',
               '일글수',
              '문장당강조',
               '문장당밑줄',
              'Jan',
              'Feb',
              'Mar',
              'Apr',
              'May',
              'June',
              'July',
              'Aug',
              'Sep',
              'Oct',
              'Nov',
              'Dec',
              '시작1일차이_T',
              '시작5일차이_T',
              '시작10일차이_T',
              '시작15일차이_T',
              '시작20일차이_T',
              '시작25일차이_T',
              '마감1일차이_T',
              '마감5일차이_T',
              '마감10일차이_T',
              '마감15일차이_T',
              '마감20일차이_T',
              '마감25일차이_T']
data_f = data[feature_names]

In [9]:
X = data_f
y = data.달성률_T

# 학습용/테스트용 데이터 분리

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [10]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000)

rf.fit(X_train, y_train)

print('훈련셋점수:',rf.score(X_train, y_train))
print('테스트셋점수:',rf.score(X_test, y_test))

y_pred = rf.predict(X_test)

import sklearn.metrics as mt

#평가 지표 값들을 계산한다.
mse = mt.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = mt.r2_score(y_test, y_pred)

print("MSE:{:.3f}\nRMSE: {:.3f}\nR2: {:.3f}".format(mse,rmse,r2))
#print(mt.accuracy_score( y_pred, y_test))
#print(mt.confusion_matrix(y_pred, y_test))

pd.DataFrame({'속성명':X.columns, '중요도':rf.feature_importances_}).sort_values('중요도', ascending=False)

훈련셋점수: 0.9600213053698861
테스트셋점수: 0.721007283986373
MSE:0.299
RMSE: 0.547
R2: 0.721


,속성명,중요도
16,펀딩성공여부,0.579809
35,일글수,0.069162
6,와디즈팔로워수,0.046817
34,일펀딩금액,0.036156
3,이미지수_T,0.021155
10,문장수_T,0.015507
5,마감배송차이_T,0.013642
0,와디즈기사개수_T,0.012839
36,문장당강조,0.012829
33,펀딩기간일_T,0.012787
